# Построение базовых решений

Мы построим три базовые модели и сравним их качество по кросс-валидации. В качестве базовых моделей выбраны:
* логистическая регрессия
* случайный лес
* градиентный бустинг

Все модели используют параметры по умолчанию, случайный лес и градиентный бустинг построены на 100 деревьях. В качестве датасета используется 80% наблюдений, разбиение ведется на 4 фолда, пропорции классов сохранены. Для базовых моделей используются только числовые признаки, пропущенные значения заменены нулем для логистической регрессии и большим числом для случайного леса и градиентного бустинга. Для логистической регрессии сделано масштабирование признаков.

In [7]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, auc, confusion_matrix

In [2]:
#importing dataset
df = pd.read_csv('train_churn.csv', header = 0)
print(df.shape)
df.head()

(32000, 231)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,Churn
0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,VOHAgOW,LM8l689qOp,NaN,NaN,Xa3G,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,Vo68gbh,NaN,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,203.0,0.0,NaN,NaN,NaN,...,NnYzgmc,NaN,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,105.0,0.0,NaN,NaN,NaN,...,ygD7j0h,LM8l689qOp,NaN,NaN,5Acm,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
4,NaN,NaN,NaN,NaN,NaN,84.0,0.0,NaN,NaN,NaN,...,smo7DIf,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


In [3]:
#creation of models and folds
scaler = StandardScaler()
skf = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 147)
lr_class = LogisticRegression()
rf_class = RandomForestClassifier(n_estimators = 100, random_state = 147)
gb_class = GradientBoostingClassifier(n_estimators = 100, random_state = 147)

In [4]:
#creation of matrix of features
X_lr = scaler.fit_transform(df.iloc[:,:190].fillna(0).values)
X_tr = df.iloc[:,:190].fillna(1e10).values
y = df['Churn'].values

In [13]:
#metrics calculation
scores = ['roc_auc', 'recall', 'precision', 'accuracy']
for score in scores:
    lr_score = np.mean(cross_val_score(lr_class, X_lr, y, cv = skf, scoring = score))
    rf_score = np.mean(cross_val_score(rf_class, X_tr, y, cv = skf, scoring = score))
    gb_score = np.mean(cross_val_score(gb_class, X_tr, y, cv = skf, scoring = score))
    print(score, lr_score, rf_score, gb_score)

roc_auc 0.626258172498 0.656112126021 0.719104194345
recall 0.0020987253962 0.000420168067227 0.00210084033613


//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


precision 0.144230769231 0.125 0.211111111111
accuracy 0.924718749415 0.925531261136 0.925125018947


In [18]:
print('accuracy of major class classifier: \n' + str(df['Churn'].value_counts() / len(df)))

accuracy of major class classifier: 
-1    0.925594
 1    0.074406
Name: Churn, dtype: float64


Выводы:
* Было построено три базовых решения, с использованием логистической регрессии, случайного леса и градиентного бустинга
* С точки зрения accuracy, полученные решения даже хуже классификации большим классом
* У нас получены очень низкие значения precision и recall, с точки зрения бизнеса, нам позволяют увеличить прибыль только правильные предсказания, если клиент разрывает контракт, а мы не предпринимаем мер по его удержанию (случай False Negative), то мы теряем прибыль, если мы предпринимаем меры по удержанию клиента, который не склонен разрывать контракт (случай False Positive), то мы тратим лишние деньги на его удержание.
* С точки зрения AUC ROC, лучшим является метод градиентного бустинга, precision и recall в данном случае также самые высокие 
* Базовые алгоритмы нуждаются в улучшении: добавление и обработка категориальных признаков, иная обработка пропущенных значений, регуляризация (в случае логистической регрессии), увеличение числа деревьв и т.д.